# __Урок 3__

## __Практическая работа__

In [1]:
# pip install tqdm

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

__Задача.__ Подойдет ли репетитор для ученика

In [3]:
X = np.array([ [   1,    1,  500,    1],
               [   1,    1,  700,    1],
               [   1,    2,  750,    2],
               [   1,    5,  600,    1],
               [   1,    3, 1450,    2],
               [   1,    0,  800,    1],
               [   1,    5, 1500,    3],
               [   1,   10, 2000,    3],
               [   1,    1,  450,    1],
               [   1,    2, 1000,    2]], dtype = np.float64)

y = np.array([0, 0, 1, 0, 1, 0, 1, 0, 1, 1])

In [4]:
def standard_scale(X):
    mean = X.mean(axis=0)
    std = X.std(axis=0)
    return (X - mean) / std

In [5]:
X_st = X.copy().astype(np.float64)
X_st[:, 1:4] = standard_scale(X_st[:, 1:4])
X_st

array([[ 1.        , -0.70710678, -0.97958969, -0.89625816],
       [ 1.        , -0.70710678, -0.56713087, -0.89625816],
       [ 1.        , -0.35355339, -0.46401617,  0.38411064],
       [ 1.        ,  0.70710678, -0.77336028, -0.89625816],
       [ 1.        ,  0.        ,  0.97958969,  0.38411064],
       [ 1.        , -1.06066017, -0.36090146, -0.89625816],
       [ 1.        ,  0.70710678,  1.08270439,  1.66447944],
       [ 1.        ,  2.47487373,  2.11385144,  1.66447944],
       [ 1.        , -0.70710678, -1.08270439, -0.89625816],
       [ 1.        , -0.35355339,  0.05155735,  0.38411064]])

1. Измените функцию calc_logloss так, чтобы нули по возможности не попадали в np.log.

In [6]:
def calc_logloss(y, y_pred):
    err = 0
    for i in range(len(y)):
        if y_pred[i] == 0:
            err += (1.0 - y[i]) * np.log(1.0 - y_pred[i])
        elif y_pred[i] == 1:
            err += y[i] * np.log(y_pred[i])
        else:
            err += y[i] * np.log(y_pred[i]) + (1.0 - y[i]) * np.log(1.0 - y_pred[i])
    return - err / len(y)

Будем путем перебора значений подбирать лучший вариант

In [9]:
y1 = np.array([1, 0])
y_pred1 = np.array([0.9, 0])
calc_logloss(y1, y_pred1)

0.05268025782891314

In [10]:
y1 = np.array([1, 0])
y_pred1 = np.array([1, 0.1])
calc_logloss(y1, y_pred1)

0.05268025782891314

In [15]:
y1 = np.array([1, 0])
y_pred1 = np.array([0.9, 0.1])
calc_logloss(y1, y_pred1)

0.10536051565782628

2. Подберите аргументы функции eval_model для логистической регрессии таким образом, чтобы log loss был минимальным.

In [16]:
def sigmoid(z):
    res = 1 / (1 + np.exp(-z))
    return res

In [17]:
def eval_model(X, y, iterations, eta=1e-4):
    np.random.seed(42)
    W = np.random.randn(X.shape[1])
    n = X.shape[0]
    
    for i in range(iterations):
        z = np.dot(X, W)
        y_pred = sigmoid(z)
        err = calc_logloss(y, y_pred)
        
        dQ = 1/n * X.T @ (y_pred - y)
        W -= eta * dQ
        if i % (iterations / 10) == 0:
            print(i, W, err)
            
    final_error = calc_logloss(y, y_pred)
    return W, final_error

In [38]:
W, error = eval_model(X_st, y, iterations=10000, eta=6)

0 [ 0.28371916 -1.76318139 -0.75287173  0.7761512 ] 0.760958797591889
1000 [  4.20570761 -11.95378336 -12.14890859  28.09152844] 0.09379547272809333
2000 [  5.58125241 -15.13678829 -16.5263895   37.15001474] 0.07468731604051156
3000 [  6.62634907 -17.33892245 -20.0948317   44.15666662] 0.06335903235940196
4000 [  7.52256211 -19.14696142 -23.21882461  50.17782245] 0.0549976773925275
5000 [  8.31574937 -20.72826932 -25.99567371  55.50605268] 0.048451000842643484
6000 [  9.02733042 -22.1439288  -28.48747432  60.28449966] 0.04318660690530466
7000 [  9.67157797 -23.42615395 -30.74202798  64.60946128] 0.03887478723378829
8000 [ 10.25924069 -24.59689562 -32.7969376   68.55352886] 0.03528957181565367
9000 [ 10.798763   -25.67283572 -34.68209997  72.1737047 ] 0.032269436325150294


3. Создайте функцию calc_pred_proba, возвращающую предсказанную вероятность класса 1. На вход подаётся W, который уже посчитан функцией eval_model, и X, на выходе — массив y_pred_proba.

In [39]:
def calc_pred_proba(W, X):
    y_pred_proba = sigmoid(np.dot(X, W))
    return y_pred_proba

In [40]:
calc_pred_proba(W, X_st)

array([1.35898579e-01, 4.70771591e-08, 1.00000000e+00, 3.60336834e-21,
       9.90291881e-01, 3.20181001e-07, 1.00000000e+00, 2.48130435e-03,
       8.70528854e-01, 1.00000000e+00])

4. Создайте функцию calc_pred, возвращающую предсказанный класс. На вход подаётся W, который уже посчитан функцией eval_model, и X, на выходе — массив y_pred.

In [41]:
def calc_pred(W, X):
    y_pred_proba = calc_pred_proba(W, X)
    y_pred = np.where(y_pred_proba > 0.5, 1, 0)
    return y_pred

In [42]:
pred = calc_pred(W, X_st)
pred

array([0, 0, 1, 0, 1, 0, 1, 0, 1, 1])